# Hypothesis Testing in Python

## Hypothesis Testing Fundamentals

### Calculating the sample mean

The `late_shipments` dataset contains supply chain data on the delivery
of medical supplies. Each row represents one delivery of a part. The
`late` columns denotes whether or not the part was delivered late. A
value of `"Yes"` means that the part was delivered late, and a value of
`"No"` means the part was delivered on time.

You'll begin your analysis by calculating a point estimate (or sample
statistic), namely the proportion of late shipments.

In `pandas`, a value's proportion in a categorical DataFrame column can
be quickly calculated using the syntax:

    prop = (df['col'] == val).mean()

`late_shipments` is available, and `pandas` is loaded as `pd`.

**Instructions**

- Print the `late_shipments` dataset.
- Calculate the proportion of late shipments in the sample; that is, the mean cases where the `late` column is `"Yes"`.

**Answer**


In [4]:
pip install pandas pyarrow

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 1.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# added/edited
import pandas as pd
late_shipments = pd.read_feather('late_shipments.feather')

In [9]:
# Print the late_shipments dataset
print(late_shipments)

# Calculate the proportion of late shipments
late_prop_samp = (late_shipments['late'] == "Yes").mean()

# Print the results
print(late_prop_samp)

          id       country managed_by  fulfill_via vendor_inco_term  \
0    36203.0       Nigeria   PMO - US  Direct Drop              EXW   
1    30998.0      Botswana   PMO - US  Direct Drop              EXW   
2    69871.0       Vietnam   PMO - US  Direct Drop              EXW   
3    17648.0  South Africa   PMO - US  Direct Drop              DDP   
4     5647.0        Uganda   PMO - US  Direct Drop              EXW   
..       ...           ...        ...          ...              ...   
995  13608.0        Uganda   PMO - US  Direct Drop              DDP   
996  80394.0    Congo, DRC   PMO - US  Direct Drop              EXW   
997  61675.0        Zambia   PMO - US  Direct Drop              EXW   
998  39182.0  South Africa   PMO - US  Direct Drop              DDP   
999   5645.0      Botswana   PMO - US  Direct Drop              EXW   

    shipment_mode  late_delivery late product_group    sub_classification  \
0             Air            1.0  Yes          HRDT              HIV t

### Calculating a z-score

Since variables have arbitrary ranges and units, we need to standardize
them. For example, a hypothesis test that gave different answers if the
variables were in Euros instead of US dollars would be of little value.
Standardization avoids that.

One standardized value of interest in a hypothesis test is called a
z-score. To calculate it, you need three numbers: the sample statistic
(point estimate), the hypothesized statistic, and the standard error of
the statistic (estimated from the bootstrap distribution).

The sample statistic is available as `late_prop_samp`.

`late_shipments_boot_distn` is a bootstrap distribution of the
proportion of late shipments, available as a list.

`pandas` and `numpy` are loaded with their usual aliases.

**Instructions**

- Hypothesize that the proportion of late shipments is 6%.
- Calculate the standard error from the standard deviation of the
  bootstrap distribution.
- Calculate the z-score.

**Answer**


In [15]:
# added/edited
import numpy as np
late_shipments_boot_distn = [
    late_shipments.sample(n=len(late_shipments), replace=True)['late_delivery'].mean()
    for _ in range(5000)
]

In [17]:
# Hypothesize that the proportion is 6%
late_prop_hyp = 0.06

# Calculate the standard error
std_error = np.std(late_shipments_boot_distn, ddof=1)

# Find z-score of late_prop_samp
z_score = (late_prop_samp - late_prop_hyp) / std_error

# Print z_score
print(z_score)


0.13000327219181468


### Calculating p-values

In order to determine whether to choose the null hypothesis or the
alternative hypothesis, you need to calculate a p-value from the
z-score.

You'll now return to the late shipments dataset and the proportion of
late shipments.

The null hypothesis, \\H\_{0}\\, is that the proportion of late
shipments is six percent.

The alternative hypothesis, \\H\_{A}\\, is that the proportion of late
shipments is **greater than** six percent.

The observed sample statistic, `late_prop_samp`, the hypothesized value,
`late_prop_hyp` (6%), and the bootstrap standard error, `std_error` are
available. `norm` from `scipy.stats` has also been loaded without an
alias.

**Instructions**

- Calculate the z-score of `late_prop_samp`.
- Calculate the p-value for the z-score, using a right-tailed test.

**Answer**


In [20]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 17.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [21]:
# added/edited
from scipy.stats import norm

In [22]:
# Calculate the z-score of late_prop_samp
z_score = (late_prop_samp - late_prop_hyp) / std_error

# Calculate the p-value
p_value = 1 - norm.cdf(z_score)
                 
# Print the p-value
print(p_value) 

0.4482819189143381


### Calculating a confidence interval

If you give a single estimate of a sample statistic, you are bound to be
wrong by some amount. For example, the hypothesized proportion of late
shipments was 6%. Even if evidence suggests the null hypothesis that the
proportion of late shipments is equal to this, for any new sample of
shipments, the proportion is likely to be a little different due to
sampling variability. Consequently, it's a good idea to state a
confidence interval. That is, you say, "we are 95% 'confident' that the
proportion of late shipments is between A and B" (for some value of A
and B).

Sampling in Python
[demonstrated](https://campus.datacamp.com/courses/sampling-in-python/bootstrap-distributions-4?ex=10)
two methods for calculating confidence intervals. Here, you'll use
quantiles of the bootstrap distribution to calculate the confidence
interval.

`late_prop_samp` and `late_shipments_boot_distn` are available; `pandas`
and `numpy` are loaded with their usual aliases.

**Instructions**

- Calculate a 95% confidence interval from `late_shipments_boot_distn`
  using the quantile method, labeling the lower and upper intervals
  `lower` and `upper`.

**Answer**


In [23]:
# Calculate 95% confidence interval using quantile method
lower = np.quantile(late_shipments_boot_distn, 0.025)
upper = np.quantile(late_shipments_boot_distn, 0.975)

# Print the confidence interval
print((lower, upper))

(0.046, 0.076)


## Two-Sample and ANOVA Tests

### Two sample mean test statistic

The hypothesis test for determining if there is a difference between the
means of two populations uses a different type of test statistic to the
z-scores you saw in Chapter 1. It's called "t", and it can be calculated
from three values from each sample using this equation.

\$\$ t = \dfrac{(\bar{x}\_{\text{child}} -
\bar{x}\_{\text{adult}})}{\sqrt{\dfrac{s\_{\text{child}}^2}{n\_{\text{child}}} +
\dfrac{s\_{\text{adult}}^2}{n\_{\text{adult}}}}} \$\$

While trying to determine why some shipments are late, you may wonder if
the weight of the shipments that were on time is **less than** the
weight of the shipments that were late. The `late_shipments` dataset has
been split into a "yes" group, where `late == "Yes"` and a "no" group
where `late == "No"`. The weight of the shipment is given in the
`weight_kilograms` variable.

The sample means for the two groups are available as `xbar_no` and
`xbar_yes`. The sample standard deviations are `s_no` and `s_yes`. The
sample sizes are `n_no` and `n_yes`. `numpy` is also loaded as `np`.

**Instructions**

- Calculate the numerator of the \\t\\ test statistic.
- Calculate the denominator of the \\t\\ test statistic.
- Use those two numbers to calculate the \\t\\ test statistic.

**Answer**


In [24]:
# Calculate the numerator of the test statistic
numerator = xbar_no - xbar_yes

# Calculate the denominator of the test statistic
denominator = np.sqrt(s_no ** 2 / n_no + s_yes ** 2 / n_yes)

# Calculate the test statistic
t_stat = numerator / denominator

# Print the test statistic
print(t_stat)


NameError: name 'xbar_no' is not defined

### From t to p

Previously, you calculated the test statistic for the two-sample problem
of whether the mean weight of shipments is smaller for shipments that
weren't late (`late == "No"`) compared to shipments that were late
(`late == "Yes"`). In order to make decisions about it, you need to
transform the test statistic with a cumulative distribution function to
get a p-value.

Recall the hypotheses:

\\H\_{0}\\: The mean weight of shipments that weren't late is the same
as the mean weight of shipments that were late.

\\H\_{A}\\: The mean weight of shipments that weren't late is less than
the mean weight of shipments that were late.

The test statistic, `t_stat`, is available, as are the samples sizes for
each group, `n_no` and `n_yes`. Use a significance level of
`alpha = 0.05`.

`t` has also been imported from `scipy.stats`.

**Instructions**

- Calculate the degrees of freedom for the test.
- Compute the p-value using the test statistic, `t_stat`.

**Answer**


In [ ]:
# Calculate the degrees of freedom
degrees_of_freedom = n_no + n_yes - 2

# Calculate the p-value from the test stat
p_value = t.cdf(t_stat, df=degrees_of_freedom)

# Print the p_value
print(p_value)


### Visualizing the difference

Before you start running hypothesis tests, it's a great idea to perform
some exploratory data analysis; that is, calculating summary statistics
and visualizing distributions.

Here, you'll look at the proportion of county-level votes for the
Democratic candidate in 2012 and 2016, `sample_dem_data`. Since the
counties are the same in both years, these samples are paired. The
columns containing the samples are `dem_percent_12` and
`dem_percent_16`.

`dem_votes_potus_12_16` is available as `sample_dem_data`. `pandas` and
`matplotlib.pyplot` are loaded with their usual aliases.

**Instructions**

- Create a new `diff` column containing the percentage of votes for the
  democratic candidate in 2012 minus the percentage of votes for the
  democratic candidate in 2016.
- Calculate the mean of the `diff` column as `xbar_diff`.
- Calculate the standard deviation of the `diff` column as `s_diff`.
- Plot a histogram of the `diff` column with 20 bins.

**Answer**


In [ ]:
# Calculate the differences from 2012 to 2016
sample_dem_data['diff'] = sample_dem_data['dem_percent_12'] - sample_dem_data['dem_percent_16']

# Print sample_dem_data
print(sample_dem_data)


# Calculate the differences from 2012 to 2016
sample_dem_data['diff'] = sample_dem_data['dem_percent_12'] - sample_dem_data['dem_percent_16']

# Find the mean of the diff column
xbar_diff = sample_dem_data['diff'].mean()

# Print xbar_diff
print(xbar_diff)


# Calculate the differences from 2012 to 2016
sample_dem_data['diff'] = sample_dem_data['dem_percent_12'] - sample_dem_data['dem_percent_16']

# Find the mean of the diff column
xbar_diff = sample_dem_data['diff'].mean()

# Find the standard deviation of the diff column
s_diff = sample_dem_data['diff'].std()

# Print s_diff
print(s_diff)


# Calculate the differences from 2012 to 2016
sample_dem_data['diff'] = sample_dem_data['dem_percent_12'] - sample_dem_data['dem_percent_16']

# Find the mean of the diff column
xbar_diff = sample_dem_data['diff'].mean()

# Find the standard deviation of the diff column
s_diff = sample_dem_data['diff'].std()

# Plot a histogram of diff with 20 bins
sample_dem_data['diff'].hist(bins=20)
plt.show()


### Using ttest()

Manually calculating test statistics and transforming them with a CDF to
get a p-value is a lot of effort to compare two sample means. The
comparison of two sample means is called a t-test, and the `pingouin`
Python package has a `.ttest()` method to accomplish it. This method
provides some flexibility in how you perform the test.

As in the previous exercise, you'll explore the difference between the
proportion of county-level votes for the Democratic candidate in 2012
and 2016 to identify if the difference is significant. The hypotheses
are as follows:

\\H\_{0}\\: The proportion of democratic votes in 2012 and 2016 were the
same. \\H\_{A}\\: The proportion of democratic votes in 2012 and 2016
were different.

`sample_dem_data` is available and has the columns `diff`,
`dem_percent_12`, and `dem_percent_16` in addition to the `state` and
`county` names. `pingouin` and has been loaded along with `pandas` as
`pd`.

**Instructions**

- Conduct a t-test on the sample differences (the `diff` column of
  `sample_dem_data`), using an appropriate alternative hypothesis chosen
  from `"two-sided"`, `"less"`, and `"greater"`.
- Conduct a paired test on the democratic votes in 2012 and 2016 (the `dem_percent_12` and `dem_percent_16` columns of `sample_dem_data`), using an appropriate alternative hypothesis.

**Answer**


In [ ]:
# Conduct a t-test on diff
test_results = pingouin.ttest(x=sample_dem_data['diff'], 
                              y=0, 
                              alternative="two-sided")
                              
# Print the test results
print(test_results)


# Conduct a t-test on diff
test_results = pingouin.ttest(x=sample_dem_data['diff'], 
                              y=0, 
                              alternative="two-sided")

# Conduct a paired t-test on dem_percent_12 and dem_percent_16
paired_test_results = pingouin.ttest(x=sample_dem_data['dem_percent_12'], 
                                     y=sample_dem_data['dem_percent_16'],
                                     paired=True,
                                     alternative="two-sided")
                              
# Print the paired test results
print(paired_test_results)


### Visualizing many categories

So far in this chapter, we've only considered the case of differences in
a numeric variable between two categories. Of course, many datasets
contain more categories. Before you get to conducting tests on many
categories, it's often helpful to perform exploratory data analysis
(EDA), calculating summary statistics for each group and visualizing the
distributions of the numeric variable for each category using box plots.

Here, we'll return to the late shipments data, and how the price of each
package (`pack_price`) varies between the three shipment modes
(`shipment_mode`): `"Air"`, `"Air Charter"`, and `"Ocean"`.

`late_shipments` is available; `pandas` and `matplotlib.pyplot` are
loaded with their standard aliases, and `seaborn` is loaded as `sns`.

**Instructions**

- Group `late_shipments` by `shipment_mode` and calculate the mean
  `pack_price` for each group, storing the result in
  `xbar_pack_by_mode`.
- Group `late_shipments` by `shipment_mode` and calculate the standard deviation `pack_price` for each group, storing the result in `s_pack_by_mode`.
- Create a boxplot from `late_shipments` with `"pack_price"` as `x` and `"shipment_mode"` as `y`.

**Answer**


In [ ]:
# Calculate the mean pack_price for each shipment_mode
xbar_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].mean()

# Print the grouped means
print(xbar_pack_by_mode)


# Calculate the mean pack_price for each shipment_mode
xbar_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].mean()

# Calculate the standard deviation of the pack_price for each shipment_mode
s_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].std()

# Print the grouped standard deviations
print(s_pack_by_mode)


# Calculate the mean pack_price for each shipment_mode
xbar_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].mean()

# Calculate the standard deviation of the pack_price for each shipment_mode
s_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].std()

# Boxplot of shipment_mode vs. pack_price
sns.boxplot(x="pack_price", y="shipment_mode", data=late_shipments)
plt.show()


### Conducting an ANOVA test

The box plots made it look like the distribution of pack price was
different for each of the three shipment modes. However, it didn't tell
us whether the mean pack price was different in each category. To
determine that, we can use an ANOVA test. The null and alternative
hypotheses can be written as follows.

\\H\_{0}\\: Pack prices for every category of shipment mode are the
same.

\\H\_{A}\\: Pack prices for some categories of shipment mode are
different.

Use a significance level of 0.1.

`late_shipments` is available and `pingouin` has been loaded.

**Instructions**

- Run an ANOVA on `late_shipments` investigating `'pack_price'` (the
  dependent variable) between the groups of `'shipment_mode'`.

**Answer**


In [ ]:
# Run an ANOVA for pack_price across shipment_mode
anova_results = pingouin.anova(data=late_shipments,
                               dv="pack_price",
                               between="shipment_mode")

# Print anova_results
print(anova_results)


### Pairwise t-tests

The ANOVA test didn't tell you which categories of shipment mode had
significant differences in pack prices. To pinpoint which categories had
differences, you could instead use pairwise t-tests.

`late_shipments` is available and `pingouin` has been loaded.

**Instructions**

- Perform pairwise t-tests on `late_shipments`'s `pack_price` variable,
  grouped by `shipment_mode`, without doing any p-value adjustment.
- Modify the pairwise t-tests to use the Bonferroni p-value adjustment.

**Answer**


In [ ]:
# Perform a pairwise t-test on pack price, grouped by shipment mode
pairwise_results = pingouin.pairwise_tests(data=late_shipments, 
                                           dv="pack_price",
                                           between="shipment_mode",
                                           padjust="none")

# Print pairwise_results
print(pairwise_results)


# Modify the pairwise t-tests to use Bonferroni p-value adjustment
pairwise_results = pingouin.pairwise_tests(data=late_shipments, 
                                           dv="pack_price",
                                           between="shipment_mode",
                                           padjust="bonf")

# Print pairwise_results
print(pairwise_results)


## Proportion Tests

### Test for single proportions

In Chapter 1, you calculated a p-value for a test hypothesizing that the
proportion of late shipments was **greater than** 6%. In that chapter,
you used a bootstrap distribution to estimate the standard error of the
statistic. An alternative is to use an equation for the standard error
based on the sample proportion, hypothesized proportion, and sample
size.

\\z = \dfrac{\hat{p} - p\_{0}}{\sqrt{\dfrac{p\_{0}\*(1-p\_{0})}{n}}}\\

You'll revisit the p-value using this simpler calculation.

`late_shipments` is available. `pandas` and `numpy` are available under
their usual aliases, and `norm` is loaded from `scipy.stats`.

**Instructions**

- Hypothesize that the proportion of late shipments is 6%.
- Calculate the sample proportion of shipments where `late` equals
  `"Yes"`.
- Calculate the number of observations in the sample.
- Calculate the numerator and denominator of the z-score.
- Calculate the z-score as the ratio of these numbers.
- Transform the z-score into a p-value, remembering that this is a "greater than" alternative hypothesis.

**Answer**


In [ ]:
# Hypothesize that the proportion of late shipments is 6%
p_0 = 0.06

# Calculate the sample proportion of late shipments
p_hat = (late_shipments['late'] == "Yes").mean()

# Calculate the sample size
n = len(late_shipments)

# Print p_hat and n
print(p_hat, n)


# Hypothesize that the proportion of late shipments is 6%
p_0 = 0.06

# Calculate the sample proportion of late shipments
p_hat = (late_shipments['late'] == "Yes").mean()

# Calculate the sample size
n = len(late_shipments)

# Calculate the numerator and denominator of the test statistic
numerator = p_hat - p_0
denominator = np.sqrt(p_0 * (1 - p_0) / n)

# Calculate the test statistic
z_score = numerator / denominator

# Print the result
print(z_score)


# Hypothesize that the proportion of late shipments is 6%
p_0 = 0.06

# Calculate the sample proportion of late shipments
p_hat = (late_shipments['late'] == "Yes").mean()

# Calculate the sample size
n = len(late_shipments)

# Calculate the numerator and denominator of the test statistic
numerator = p_hat - p_0
denominator = np.sqrt(p_0 * (1 - p_0) / n)

# Calculate the test statistic
z_score = numerator / denominator

# Calculate the p-value from the z-score
p_value = 1 - norm.cdf(z_score)

# Print the p-value
print(p_value)


### Test of two proportions

You may wonder if the amount paid for freight affects whether or not the
shipment was late. Recall that in the `late_shipments` dataset, whether
or not the shipment was late is stored in the `late` column. Freight
costs are stored in the `freight_cost_group` column, and the categories
are `"expensive"` and `"reasonable"`.

The hypotheses to test, with `"late"` corresponding to the proportion of
late shipments for that group, are

\\H\_{0}\\: \\late\_{\text{expensive}} - late\_{\text{reasonable}} = 0\\

\\H\_{A}\\: \\late\_{\text{expensive}} - late\_{\text{reasonable}} \>
0\\

`p_hats` contains the estimates of population proportions (sample
proportions) for each `freight_cost_group`:

    freight_cost_group  late
    expensive           Yes     0.082569
    reasonable          Yes     0.035165
    Name: late, dtype: float64

`ns` contains the sample sizes for these groups:

    freight_cost_group
    expensive     545
    reasonable    455
    Name: late, dtype: int64

`pandas` and `numpy` have been imported under their usual aliases, and
`norm` is available from `scipy.stats`.

- Calculate the p-value from the z-score.

**Instructions**

- Calculate the pooled sample proportion, \\\hat{p}\\, from `p_hats` and
  `ns`.

\$\$ \hat{p} = \frac{n\_{\text{expensive}} \times
\hat{p}\_{\text{expensive}} + n\_{\text{reasonable}} \times
\hat{p}\_{\text{reasonable}}}{n\_{\text{expensive}} +
n\_{\text{reasonable}}} \$\$

Calculate the standard error of the sample *using this equation.*

$$\text{SE}({\hat{p}}\_{\text{expensive}} - {\hat{p}}\_{\text{reasonable}}) = \sqrt{\frac{\hat{p} \times (1 - \hat{p})}{n\_{\text{expensive}}} + \frac{\hat{p} \times (1 - \hat{p})}{n\_{\text{reasonable}}}}$$

- Calculate `p_hat` multiplied by `(1 - p_hat)`.
- Divide `p_hat_times_not_p_hat` by the number of `"reasonable"` rows
    and by the number of `"expensive"` rows, and sum those two values.
- Calculate `std_error` by taking the square root of
    `p_hat_times_not_p_hat_over_ns`.

-   Calculate the z-score *using the following equation.*

$$z = \frac{({\hat{p}}\_{\text{expensive}} - {\hat{p}}\_{\text{reasonable}})}{\text{SE}({\hat{p}}\_{\text{expensive}} - {\hat{p}}\_{\text{reasonable}})}$$

**Answer**


In [ ]:
# Calculate the pooled estimate of the population proportion
p_hat = (p_hats["reasonable"] * ns["reasonable"] + p_hats["expensive"] * ns["expensive"]) / (ns["reasonable"] + ns["expensive"])

# Print the result
print(p_hat)


# Calculate the pooled estimate of the population proportion
p_hat = (p_hats["reasonable"] * ns["reasonable"] + p_hats["expensive"] * ns["expensive"]) / (ns["reasonable"] + ns["expensive"])

# Calculate p_hat one minus p_hat
p_hat_times_not_p_hat = p_hat * (1 - p_hat)

# Divide this by each of the sample sizes and then sum
p_hat_times_not_p_hat_over_ns = p_hat_times_not_p_hat / ns["expensive"] + p_hat_times_not_p_hat / ns["reasonable"]

# Calculate the standard error
std_error = np.sqrt(p_hat_times_not_p_hat_over_ns)

# Print the result
print(std_error)


# Calculate the pooled estimate of the population proportion
p_hat = (p_hats["reasonable"] * ns["reasonable"] + p_hats["expensive"] * ns["expensive"]) / (ns["reasonable"] + ns["expensive"])

# Calculate p_hat one minus p_hat
p_hat_times_not_p_hat = p_hat * (1 - p_hat)

# Divide this by each of the sample sizes and then sum
p_hat_times_not_p_hat_over_ns = p_hat_times_not_p_hat / ns["expensive"] + p_hat_times_not_p_hat / ns["reasonable"]

# Calculate the standard error
std_error = np.sqrt(p_hat_times_not_p_hat_over_ns)

# Calculate the z-score
z_score = (p_hats["expensive"] - p_hats["reasonable"]) / std_error

# Print z_score
print(z_score)


# Calculate the pooled estimate of the population proportion
p_hat = (p_hats["reasonable"] * ns["reasonable"] + p_hats["expensive"] * ns["expensive"]) / (ns["reasonable"] + ns["expensive"])

# Calculate p_hat one minus p_hat
p_hat_times_not_p_hat = p_hat * (1 - p_hat)

# Divide this by each of the sample sizes and then sum
p_hat_times_not_p_hat_over_ns = p_hat_times_not_p_hat / ns["expensive"] + p_hat_times_not_p_hat / ns["reasonable"]

# Calculate the standard error
std_error = np.sqrt(p_hat_times_not_p_hat_over_ns)

# Calculate the z-score
z_score = (p_hats["expensive"] - p_hats["reasonable"]) / std_error

# Calculate the p-value from the z-score
p_value = 1 - norm.cdf(z_score)

# Print p_value
print(p_value)


### proportions_ztest() for two samples

That took a lot of effort to calculate the p-value, so while it is
useful to see how the calculations work, it isn't practical to do in
real-world analyses. For daily usage, it's better to use the
`statsmodels` package.

Recall the hypotheses.

\\H\_{0}\\: \\late\_{\text{expensive}} - late\_{\text{reasonable}} = 0\\

\\H\_{A}\\: \\late\_{\text{expensive}} - late\_{\text{reasonable}} \>
0\\

`late_shipments` is available, containing the `freight_cost_group`
column. `numpy` and `pandas` have been loaded under their standard
aliases, and `proportions_ztest` has been loaded from
`statsmodels.stats.proportion`.

**Instructions**

- Get the counts of the `late` column grouped by `freight_cost_group`.
- Extract the number of `"Yes"`'s for the two `freight_cost_group` into a `numpy` array, specifying the `'expensive'` count and then `'reasonable'`.
- Determine the overall number of rows in each `freight_cost_group` as a `numpy` array, specifying the `'expensive'` count and then `'reasonable'`.
- Run a z-test using `proportions_ztest()`, specifying `alternative` as `"larger"`.

**Answer**


In [ ]:
# Count the late column values for each freight_cost_group
late_by_freight_cost_group = late_shipments.groupby("freight_cost_group")['late'].value_counts()

# Print the counts
print(late_by_freight_cost_group)


# Count the late column values for each freight_cost_group
late_by_freight_cost_group = late_shipments.groupby("freight_cost_group")['late'].value_counts()

# Create an array of the "Yes" counts for each freight_cost_group
success_counts = np.array([45, 16])

# Create an array of the total number of rows in each freight_cost_group
n = np.array([45 + 500, 16 + 439])

# Run a z-test on the two proportions
stat, p_value = proportions_ztest(count=success_counts, nobs=n,
                                  alternative="larger")

# Print the results
print(stat, p_value)


### Performing a chi-square test

The *chi-square independence test* compares proportions of successes of
one categorical variable across the categories of another categorical
variable.

Trade deals often use a form of business shorthand in order to specify
the exact details of their contract. These are International Chamber of
Commerce (ICC) international commercial terms, or *incoterms* for short.

The `late_shipments` dataset includes a `vendor_inco_term` that
describes the incoterms that applied to a given shipment. The choices
are:

- [`EXW`](https://www.investopedia.com/terms/e/exw.asp): "Ex works". The
  buyer pays for transportation of the goods.
- [`CIP`](https://www.investopedia.com/terms/c/carriage-and-insurance-paid-cip.asp):
  "Carriage and insurance paid to". The seller pays for freight and
  insurance until the goods board a ship.
- [`DDP`](https://www.investopedia.com/terms/d/delivery-duty-paid.asp):
  "Delivered duty paid". The seller pays for transportation of the goods
  until they reach a destination port.
- [`FCA`](https://www.investopedia.com/terms/f/fca.asp): "Free carrier".
  The seller pays for transportation of the goods.

Perhaps the incoterms affect whether or not the freight costs are
expensive. Test these hypotheses with a significance level of `0.01`.

\\H\_{0}\\: `vendor_inco_term` and `freight_cost_group` are independent.

\\H\_{A}\\: `vendor_inco_term` and `freight_cost_group` are associated.

`late_shipments` is available, and the following have been loaded:
`matplotlib.pyplot` as `plt`, `pandas` as `pd`, and `pingouin`.

**Instructions**

- Calculate the proportion of `freight_cost_group` in `late_shipments`
  grouped by `vendor_inco_term`.
- Unstack the `.value_counts()` result to be in wide format instead of long.
- Create a proportional stacked bar plot with bars filled based on `freight_cost_group` across the levels of `vendor_inco_term`.
- Perform a chi-square test of independence on `freight_cost_group` and `vendor_inco_term` in the `late_shipments` dataset.

**Answer**


In [ ]:
# Proportion of freight_cost_group grouped by vendor_inco_term
props = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts(normalize=True)

# Print props
print(props)


# Proportion of freight_cost_group grouped by vendor_inco_term
props = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts(normalize=True)

# Convert props to wide format
wide_props = props.unstack()

# Print wide_props
print(wide_props)


# Proportion of freight_cost_group grouped by vendor_inco_term
props = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts(normalize=True)

# Convert props to wide format
wide_props = props.unstack()

# Proportional stacked bar plot of freight_cost_group vs. vendor_inco_term
wide_props.plot(kind="bar", stacked=True)
plt.show()


# Proportion of freight_cost_group grouped by vendor_inco_term
props = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts(normalize=True)

# Convert props to wide format
wide_props = props.unstack()

# Proportional stacked bar plot of freight_cost_group vs. vendor_inco_term
wide_props.plot(kind="bar", stacked=True)
plt.show()

# Determine if freight_cost_group and vendor_inco_term are independent
expected, observed, stats = pingouin.chi2_independence(data=late_shipments, x="vendor_inco_term", y="freight_cost_group")

# Print results
print(stats[stats['test'] == 'pearson']) 


### Visualizing goodness of fit

The *chi-square goodness of fit test* compares proportions of each level
of a categorical variable to hypothesized values. Before running such a
test, it can be helpful to visually compare the distribution in the
sample to the hypothesized distribution.

Recall the vendor incoterms in the `late_shipments` dataset. You
hypothesize that the four values occur with these frequencies in the
population of shipments.

- `CIP`: 0.05
- `DDP`: 0.1
- `EXW`: 0.75
- `FCA`: 0.1

These frequencies are stored in the `hypothesized` DataFrame.

The `incoterm_counts` DataFrame stores the `.value_counts()` of the
`vendor_inco_term` column.

`late_shipments` is available; `pandas` and `matplotlib.pyplot` are
loaded with their standard aliases.

**Instructions**

- Find the total number of rows in `late_shipments`.
- Add a column named `n` to the `hypothesized` DataFrame that is the `hypothesized` `prop` column times `n_total`.
- Create a bar graph of `'n'` versus `'vendor_inco_term'` for the `incoterm_counts` data, specifying a red color.
- Add blue bars to the plot showing the same results from the `hypothesized` DataFrame, specifying an `alpha` of `0.5`.

**Answer**


In [ ]:
# Find the number of rows in late_shipments
n_total = len(late_shipments)

# Print n_total
print(n_total)


# Find the number of rows in late_shipments
n_total = len(late_shipments)

# Create n column that is prop column * n_total
hypothesized["n"] = hypothesized["prop"] * n_total

# Print the modified hypothesized DataFrame
print(hypothesized)


# Find the number of rows in late_shipments
n_total = len(late_shipments)

# Create n column that is prop column * n_total
hypothesized["n"] = hypothesized["prop"] * n_total

# Plot a red bar graph of n vs. vendor_inco_term for incoterm_counts
plt.bar(incoterm_counts['vendor_inco_term'], incoterm_counts['n'], color="red", label="Observed")
plt.legend()
plt.show()


# Find the number of rows in late_shipments
n_total = len(late_shipments)

# Create n column that is prop column * n_total
hypothesized["n"] = hypothesized["prop"] * n_total

# Plot a red bar graph of n vs. vendor_inco_term for incoterm_counts
plt.bar(incoterm_counts['vendor_inco_term'], incoterm_counts['n'], color="red", label="Observed")

# Add a blue bar plot for the hypothesized counts
plt.bar(hypothesized['vendor_inco_term'], hypothesized['n'], alpha=0.5, color="blue", label="Hypothesized")
plt.legend()
plt.show()


### Performing a goodness of fit test

The bar plot of `vendor_inco_term` suggests that the distribution across
the four categories was quite close to the hypothesized distribution.
You'll need to perform a *chi-square goodness of fit test* to see
whether the differences are statistically significant.

Recall the hypotheses for this type of test:

\\H\_{0}\\: The sample matches with the hypothesized distribution.

\\H\_{A}\\: The sample does not match with the hypothesized
distribution.

To decide which hypothesis to choose, we'll set a significance level of
`0.1`.

`late_shipments`, `incoterm_counts`, and `hypothesized` from the last
exercise are available. `chisquare` from `scipy.stats` has been loaded.

**Instructions**

- Using the `incoterm_counts` and `hypothesized` datasets, perform a
  chi-square goodness of fit test on the incoterm counts, `n`.

**Answer**


In [ ]:
# Perform a goodness of fit test on the incoterm counts n
gof_test = chisquare(f_obs=incoterm_counts['n'], 
                     f_exp=hypothesized['n'])

# Print gof_test results
print(gof_test)


## Non-Parametric Tests

### Testing sample size

In order to conduct a hypothesis test and be sure that the result is
fair, a sample must meet three requirements: it is a random sample of
the population, the observations are independent, and there are enough
observations. Of these, only the last condition is easily testable with
code.

The minimum sample size depends on the type of hypothesis tests you want
to perform. You'll now test some scenarios on the `late_shipments`
dataset.

Note that the `.all()` method from `pandas` can be used to check if all
elements are true. For example, given a DataFrame `df` with numeric
entries, you check to see if all its elements are less than `5`, using
`(df < 5).all()`.

`late_shipments` is available, and `pandas` is loaded as `pd`.

**Instructions**

- Get the count of each value in the `freight_cost_group` column of
  `late_shipments`.
- Insert a suitable number to inspect whether the counts are "big
  enough" for a two sample t-test.

<!-- -->

- Get the count of each value in the `late` column of `late_shipments`.
- Insert a suitable number to inspect whether the counts are "big
  enough" for a one sample proportion test.

<!-- -->

- Get the count of each value in the `freight_cost_group` column of
  `late_shipments` grouped by `vendor_inco_term`.
- Insert a suitable number to inspect whether the counts are "big
  enough" for a chi-square independence test.

<!-- -->

- Get the count of each value in the `shipment_mode` column of
  `late_shipments`.
- Insert a suitable number to inspect whether the counts are "big
  enough" for an ANOVA test.

**Answer**


In [ ]:
# Count the freight_cost_group values
counts = late_shipments['freight_cost_group'].value_counts()

# Print the result
print(counts)

# Inspect whether the counts are big enough
print((counts >= 30).all())


# Count the late values
counts = late_shipments['late'].value_counts()

# Print the result
print(counts)

# Inspect whether the counts are big enough
print((counts >= 10).all())


# Count the values of freight_cost_group grouped by vendor_inco_term
counts = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts()

# Print the result
print(counts)

# Inspect whether the counts are big enough
print((counts >= 5).all())


# Count the shipment_mode values
counts = late_shipments['shipment_mode'].value_counts()

# Print the result
print(counts)

# Inspect whether the counts are big enough
print((counts >= 30).all())


### Wilcoxon signed-rank test

You'll explore the difference between the proportion of county-level
votes for the Democratic candidate in 2012 and 2016 to identify if the
difference is significant.

`sample_dem_data` is available, and has columns `dem_percent_12` and
`dem_percent_16` in addition to `state` and `county` names. The
following packages have also been loaded: `pingouin` and `pandas` as
`pd`.

**Instructions**

- Conduct a paired t-test on the percentage columns using an appropriate
  alternative hypothesis.

<!-- -->

- Conduct a Wilcoxon-signed rank test on the same columns.

**Answer**


In [ ]:
# Conduct a paired t-test on dem_percent_12 and dem_percent_16
paired_test_results = pingouin.ttest(x=sample_dem_data['dem_percent_12'], 
                                     y=sample_dem_data['dem_percent_16'],
                                     paired=True,
                                     alternative="two-sided")

# Print paired t-test results
print(paired_test_results)


# Conduct a Wilcoxon test on dem_percent_12 and dem_percent_16
wilcoxon_test_results = pingouin.wilcoxon(x=sample_dem_data['dem_percent_12'], 
                                          y=sample_dem_data['dem_percent_16'],
                                          alternative="two-sided")

# Print Wilcoxon test results
print(wilcoxon_test_results)


### Wilcoxon-Mann-Whitney

Another class of non-parametric hypothesis tests are called *rank sum
tests*. Ranks are the positions of numeric values from smallest to
largest. Think of them as positions in running events: whoever has the
fastest (smallest) time is rank 1, second fastest is rank 2, and so on.

By calculating on the ranks of data instead of the actual values, you
can avoid making assumptions about the distribution of the test
statistic. It's more robust in the same way that a median is more robust
than a mean.

One common rank-based test is the Wilcoxon-Mann-Whitney test, which is
like a non-parametric t-test.

`late_shipments` is available, and the following packages have been
loaded: `pingouin` and `pandas` as `pd`.

**Instructions**

- Select `weight_kilograms` and `late` from `late_shipments`, assigning
  the name `weight_vs_late`.
- Convert `weight_vs_late` from long-to-wide format, setting `columns`
  to `'late'`.
- Run a Wilcoxon-Mann-Whitney test for a difference in
  `weight_kilograms` when the shipment was late and on-time.

**Answer**


In [ ]:
# Select the weight_kilograms and late columns
weight_vs_late = late_shipments[["weight_kilograms", "late"]]

# Convert weight_vs_late into wide format
weight_vs_late_wide = weight_vs_late.pivot(columns='late', 
                                           values='weight_kilograms')

# Run a two-sided Wilcoxon-Mann-Whitney test on weight_kilograms vs. late
wmw_test = pingouin.mwu(x=weight_vs_late_wide['No'],
                        y=weight_vs_late_wide['Yes'],
                        alternative='two-sided')

# Print the test results
print(wmw_test)


### Kruskal-Wallis

Recall that the Kruskal-Wallis test is a non-parametric version of an
ANOVA test, comparing the means across multiple groups.

`late_shipments` is available, and the following packages have been
loaded: `pingouin` and `pandas` as `pd`.

**Instructions**

- Run a Kruskal-Wallis test on `weight_kilograms` between the different
  shipment modes in `late_shipments`.

**Answer**


In [ ]:
# Run a Kruskal-Wallis test on weight_kilograms vs. shipment_mode
kw_test = pingouin.kruskal(data=late_shipments, 
                           dv='weight_kilograms',
                           between='shipment_mode')

# Print the results
print(kw_test)
